In [24]:
# Import the necessary libraries, including pyodbc and pandas
import pyodbc
import pandas as pd
import os
import re

# Preparations

## Installations

In [49]:
# Install the necessary libraries using pip

# !pip install pyodbc pandas

## Set Up Database Connection
Set up the connection to the MSSQL 2012 database using pyodbc.connect().

In [41]:
! echo "check if VPN is connected..."
! scutil --nc status "VPN" | head -n 1 | grep Connected || (echo "VPN is not connected"; exit 1)

check if VPN is connected...
Connected


In [30]:
def get_linux_password(env_file):
    env_file = os.path.expanduser(env_file)
    with open(env_file, "r") as file:
        for line in file:
            match = re.match(r"LINUX_PASSWORD='''(.+)'''", line)
            if match:
                return match.group(1)
    return None

db_password = get_linux_password("~/.bitwarden1.env")


In [36]:
# Define the connection string
connection_string = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=10.213.254.250\\TEST01;"
    "DATABASE=ITST_Test_1;"
    "UID=sa;"
    f"PWD={db_password}"
)

# Establish the connection
connection = pyodbc.connect(connection_string)

# Verify the connection by printing the server name
cursor = connection.cursor()
cursor.execute("SELECT @@SERVERNAME")
server_name = cursor.fetchone()[0]
print(f"Connected to server: {server_name}")


Connected to server: LAB-SQL-01\TEST01


In [50]:
# cursor.close()
# connection.close()

In [52]:
# Execute a Simple Query
def run_query(query):    
    # Execute a simple SQL query
    cursor.execute(query)
    
    # Fetch the results
    results = cursor.fetchall()
    
    # Convert the results to a pandas DataFrame
    df = pd.DataFrame.from_records(results, columns=[column[0] for column in cursor.description])
    
    return df

In [48]:
run_query("SELECT TOP 10 * FROM INFORMATION_SCHEMA.TABLES")

# Check long qureis
